# Data Transformation for Redshift

## Shelter Data

In [246]:
shelter_df = pd.read_csv(os.path.abspath(os.getcwd())+'\\Data\\daily-shelter-overnight-service-occupancy-capacity-2021.csv')

In [247]:
shelter_df['date'] = pd.to_datetime(shelter_df['OCCUPANCY_DATE'].apply(lambda x: "{}{}".format('20', x)))

In [248]:
shelter_df.head()

,_id,OCCUPANCY_DATE,ORGANIZATION_ID,ORGANIZATION_NAME,SHELTER_ID,SHELTER_GROUP,LOCATION_ID,LOCATION_NAME,LOCATION_ADDRESS,LOCATION_POSTAL_CODE,LOCATION_CITY,LOCATION_PROVINCE,PROGRAM_ID,PROGRAM_NAME,SECTOR,PROGRAM_MODEL,OVERNIGHT_SERVICE_TYPE,PROGRAM_AREA,SERVICE_USER_COUNT,CAPACITY_TYPE,CAPACITY_ACTUAL_BED,CAPACITY_FUNDING_BED,OCCUPIED_BEDS,UNOCCUPIED_BEDS,UNAVAILABLE_BEDS,CAPACITY_ACTUAL_ROOM,CAPACITY_FUNDING_ROOM,OCCUPIED_ROOMS,UNOCCUPIED_ROOMS,UNAVAILABLE_ROOMS,OCCUPANCY_RATE_BEDS,OCCUPANCY_RATE_ROOMS,date
0,1,21-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1103.0,COSTI/City North York West Hotel Program,1677 Wilson Ave,M3L 1A5,North York,ON,15371,COSTI North York West Hotel - Family Program,Families,Emergency,Motel/Hotel Shelter,COVID-19 Response,74,Room Based Capacity,NaN,NaN,NaN,NaN,NaN,29.0,58.0,26.0,3.0,29.0,NaN,89.66,2021-01-01
1,2,21-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1103.0,COSTI/City North York West Hotel Program,1677 Wilson Ave,M3L 1A5,North York,ON,16211,COSTI North York West Hotel - Seniors Program,Mixed Adult,Emergency,Motel/Hotel Shelter,COVID-19 Response,3,Room Based Capacity,NaN,NaN,NaN,NaN,NaN,3.0,0.0,3.0,0.0,0.0,NaN,100.00,2021-01-01
2,3,21-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1103.0,COSTI/City North York West Hotel Program,1677 Wilson Ave,M3L 1A5,North York,ON,16192,COSTI North York West Hotel Program - Men,Men,Emergency,Motel/Hotel Shelter,COVID-19 Response,24,Room Based Capacity,NaN,NaN,NaN,NaN,NaN,28.0,0.0,23.0,5.0,0.0,NaN,82.14,2021-01-01
3,4,21-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1103.0,COSTI/City North York West Hotel Program,1677 Wilson Ave,M3L 1A5,North York,ON,16191,COSTI North York West Hotel Program - Mixed Adult,Mixed Adult,Emergency,Motel/Hotel Shelter,COVID-19 Response,25,Room Based Capacity,NaN,NaN,NaN,NaN,NaN,17.0,0.0,17.0,0.0,0.0,NaN,100.00,2021-01-01
4,5,21-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1103.0,COSTI/City North York West Hotel Program,1677 Wilson Ave,M3L 1A5,North York,ON,16193,COSTI North York West Hotel Program - Women,Women,Emergency,Motel/Hotel Shelter,COVID-19 Response,13,Room Based Capacity,NaN,NaN,NaN,NaN,NaN,14.0,0.0,13.0,1.0,0.0,NaN,92.86,2021-01-01


In [267]:
# grouping shelter data to get daily averages for each postal code location
shelter_groupby_df = shelter_df.groupby(
    ['date', 'LOCATION_POSTAL_CODE']
).agg(
    {
        'ORGANIZATION_ID': "count",    #  get the count of organizations
        'OCCUPANCY_RATE_BEDS': "mean",  # get the average of rate of beds occupied
        'OCCUPANCY_RATE_ROOMS': "mean",  # get the average rate of rooms occupied
        'SERVICE_USER_COUNT': "mean",  # get the average service users
        'CAPACITY_ACTUAL_BED': "mean",  # get the average capacity of beds
        'CAPACITY_ACTUAL_ROOM': "mean"  # get the average capacity of rooms
    }
).reset_index(drop=False)

In [268]:
# shelter_groupby_df.reset_index(drop=False).to_csv('shelter_test_data.csv', index=None)

In [269]:
# getting the first 3 characters in postal code
shelter_groupby_df['postal_code'] = shelter_groupby_df['LOCATION_POSTAL_CODE'].str.split(' ', n = 1, expand = True)[0]

In [270]:
shelter_groupby_df

,date,LOCATION_POSTAL_CODE,ORGANIZATION_ID,OCCUPANCY_RATE_BEDS,OCCUPANCY_RATE_ROOMS,SERVICE_USER_COUNT,CAPACITY_ACTUAL_BED,CAPACITY_ACTUAL_ROOM,postal_code
0,2021-01-01,L4L 8P5,1,NaN,100.0,67.0,NaN,51.0,L4L
1,2021-01-01,M1E 2M6,2,91.670,100.0,33.5,12.0,16.0,M1E
2,2021-01-01,M1E 2N8,1,NaN,100.0,37.0,NaN,18.0,M1E
3,2021-01-01,M1E 2P4,1,NaN,76.0,26.0,NaN,25.0,M1E
4,2021-01-01,M1E 2P9,2,NaN,100.0,26.0,NaN,12.0,M1E
...,...,...,...,...,...,...,...,...,...
35049,2021-12-31,M6R 2K3,1,NaN,100.0,38.0,NaN,38.0,M6R
35050,2021-12-31,M8Y 1A3,1,100.000,NaN,18.0,18.0,NaN,M8Y
35051,2021-12-31,M9V 1P9,2,96.430,NaN,18.0,18.5,NaN,M9V
35052,2021-12-31,M9V 2X6,2,96.985,NaN,32.5,33.5,NaN,M9V


## Inside Airbnb Listing Data

In [271]:
listing_df_09 = pd.read_csv(os.path.abspath(os.getcwd())+'\\Data\\listings-2021-09-07.csv')
listing_df_09['date'] = pd.to_datetime(listing_df_09['date'])
listing_df_12 = pd.read_csv(os.path.abspath(os.getcwd())+'\\Data\\listings-2021-12-05.csv')
listing_df_12['date'] = pd.to_datetime(listing_df_12['date'])

In [272]:
listing_df = pd.concat([listing_df_09, listing_df_12])

In [273]:
# getting only listings that are registered with ML&S
reg_listing_df = listing_df[listing_df['license'].notnull()]

In [274]:
# cleaning up licenses: remove spaces & make uppercase
reg_listing_df['license'] = reg_listing_df['license'].replace('\s+', ' ', regex=True).str.upper()

<ipython-input-274-dd1637230873>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_listing_df['license'] = reg_listing_df['license'].replace('\s+', ' ', regex=True).str.upper()


In [275]:
reg_listing_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,date
10,43964,Bright entire 2-bedrm basement suite private e...,192364,Mitra,NaN,Wexford/Maryvale,43.74922,-79.29188,Private room,93,2,31,2019-08-05,0.58,1,345,1,STR-2012-FMQPHC,2021-09-07
28,96336,John & Bren's Queen West 3 bedroom townhouse,511506,John,NaN,Niagara,43.64399,-79.40966,Entire home/apt,239,4,126,2019-12-29,1.53,1,109,4,STR-2009-FXRRPD,2021-09-07
29,104096,Hip and modern family-friendly home,349172,Avery,NaN,South Riverdale,43.66406,-79.32749,Entire home/apt,175,5,18,2016-09-06,0.15,1,280,0,STR-2012-HCVDVJ,2021-09-07
36,148945,Downtown Toronto Near Lake Ontario,718884,John,NaN,Waterfront Communities-The Island,43.64825,-79.36466,Private room,82,3,27,2015-06-17,0.35,1,250,0,STR-2010-FJCKPB,2021-09-07
42,192860,"Modern, Clean & Quiet 2bd @ High Park",533496,Paul,NaN,Junction Area,43.66492,-79.47384,Entire home/apt,135,3,109,2018-11-07,0.95,1,77,3,STR-2010-FPPDPC,2021-09-07


## Registered STR Permits

In [276]:
mls_df = pd.read_csv(os.path.abspath(os.getcwd())+'\\Data\\Municipal Licensing & Standards\\short-term-rental-registrations-data.csv')

In [277]:
mls_df

,_id,operator_registration_number,postal_code,ward_number
0,491448,STR-2008-FMKPBC,M4T,11.0
1,491449,STR-2009-FVRPBD,M6G,11.0
2,491450,STR-2009-GCXPBF,M1L,20.0
3,491451,STR-2009-GLDPBG,M6J,9.0
4,491452,STR-2009-GTKPBH,M6J,11.0
...,...,...,...,...
3429,494877,STR-2106-GCDXVD,M4L,14.0
3430,494878,STR-2106-GSRXVG,M2K,17.0
3431,494879,STR-2106-GZXXVH,M5A,13.0
3432,494880,STR-2106-HRKXVK,M1T,22.0


## Combining Data Sources

In [278]:
# merge Inside Airbnb listings with registered STR list
combined_str_df = pd.merge(reg_listing_df, mls_df, left_on=['license'], right_on=['operator_registration_number'], how='left')

In [279]:
combined_str_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,date,_id,operator_registration_number,postal_code,ward_number
0,43964,Bright entire 2-bedrm basement suite private e...,192364,Mitra,NaN,Wexford/Maryvale,43.749220,-79.291880,Private room,93,2,31,2019-08-05,0.58,1,345,1,STR-2012-FMQPHC,2021-09-07,493529.0,STR-2012-FMQPHC,M1R,21.0
1,96336,John & Bren's Queen West 3 bedroom townhouse,511506,John,NaN,Niagara,43.643990,-79.409660,Entire home/apt,239,4,126,2019-12-29,1.53,1,109,4,STR-2009-FXRRPD,2021-09-07,491666.0,STR-2009-FXRRPD,M6J,10.0
2,104096,Hip and modern family-friendly home,349172,Avery,NaN,South Riverdale,43.664060,-79.327490,Entire home/apt,175,5,18,2016-09-06,0.15,1,280,0,STR-2012-HCVDVJ,2021-09-07,493097.0,STR-2012-HCVDVJ,M4L,14.0
3,148945,Downtown Toronto Near Lake Ontario,718884,John,NaN,Waterfront Communities-The Island,43.648250,-79.364660,Private room,82,3,27,2015-06-17,0.35,1,250,0,STR-2010-FJCKPB,2021-09-07,492133.0,STR-2010-FJCKPB,M5A,10.0
4,192860,"Modern, Clean & Quiet 2bd @ High Park",533496,Paul,NaN,Junction Area,43.664920,-79.473840,Entire home/apt,135,3,109,2018-11-07,0.95,1,77,3,STR-2010-FPPDPC,2021-09-07,492193.0,STR-2010-FPPDPC,M6P,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,53674054,"A1-Affordable-500 Mbit WIFI, Subway 9 min.",431817031,Sahra,NaN,York University Heights,43.750932,-79.503647,Private room,69,1,0,NaN,NaN,6,6,0,STR-2101-FJWPHB,2021-12-05,494098.0,STR-2101-FJWPHB,M3J,7.0
9565,53674647,HEART OF TORONTO! ACROSS CN TOWER & MTCC!,154399618,Dale,NaN,Waterfront Communities-The Island,43.644698,-79.390476,Entire home/apt,104,1,0,NaN,NaN,14,21,0,STR-2011-FJLXVB,2021-12-05,492680.0,STR-2011-FJLXVB,M5V,10.0
9566,53675452,Hwy401/Keele Cozy bedroom with private bathroom,381313504,Grace,NaN,Maple Leaf,43.719152,-79.484080,Private room,42,1,0,NaN,NaN,3,71,0,STR-2009-KJFGWW,2021-12-05,NaN,NaN,NaN,NaN
9567,53676487,"""The Red"" Downtown Toronto 5 Star Location",311812420,Brian,NaN,Kensington-Chinatown,43.651695,-79.399825,Private room,40,7,0,NaN,NaN,1,78,0,STR-2013-JKFASD,2021-12-05,NaN,NaN,NaN,NaN


In [280]:
# group by postal code: get the number of listings by registered operators, the average number of listing per host, 
# the average price, the average min. of nights, the average num. of reviews per month, and average availabiliy per year
str_groupby_df = combined_str_df.groupby(
    ['date','postal_code']
).agg(
    {
        'operator_registration_number': "count",    #  get the count of listings
        'calculated_host_listings_count': "mean",  # get the average listings per host
        'price': "mean",  # get the average rate
        'minimum_nights': "mean",  # get the average number of min. nights
        'reviews_per_month': "mean",  # get the average number of reviews per month
        'availability_365': "mean"  # get the average availability per year
    }
).reset_index(drop=False)

In [287]:
# merge Inside Airbnb listings & registered STR list with shelter data
pd.merge_asof(shelter_groupby_df, str_groupby_df, on="date",
                                 by="postal_code").to_csv('final_df.csv', index=None)